In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
x = data['review']
y = data['sentiment']

In [5]:
y = y.map({'positive': 1, 'negative': 0})

In [6]:
from bs4 import BeautifulSoup
def clean_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    return cleaned_text

In [7]:
x = x.apply(clean_text)

In [8]:
x.head()

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. The filming tec...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object

In [9]:
x = x.str.lower()
x.head()

0    one of the other reviewers has mentioned that ...
1    a wonderful little production. the filming tec...
2    i thought this was a wonderful way to spend ti...
3    basically there's a family where a little boy ...
4    petter mattei's "love in the time of money" is...
Name: review, dtype: object

In [10]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

/Users/prachyasumandas/Library/Python/3.11/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [12]:
print(type(x_train))
print(x_train[:5])

<class 'pandas.core.series.Series'>
39087    that's what i kept asking myself during the ma...
30893    i did not watch the entire movie. i could not ...
45278    a touching love story reminiscent of in the m...
16398    this latter-day fulci schlocker is a totally a...
13653    first of all, i firmly believe that norwegian ...
Name: review, dtype: object


In [13]:
x_train = x_train.astype(str)
x_test = x_test.astype(str)

In [14]:
x.count()

50000

In [15]:
print(type(x_train))
len(x_train)

<class 'pandas.core.series.Series'>


40000

In [16]:
# Remove NaN first
x_train = x_train.fillna("")
x_test  = x_test.fillna("")

# Force convert each element to string
x_train = x_train.apply(lambda x: str(x))
x_test  = x_test.apply(lambda x: str(x))

# Convert to list
x_train = x_train.tolist()
x_test  = x_test.tolist()

In [17]:
MAX_LEN = 128

train_encodings = tokenizer(
    x_train,
    truncation=True,
    padding=True,
    max_length=MAX_LEN
)

test_encodings = tokenizer(
    x_test,
    truncation=True,
    padding=True,
    max_length=MAX_LEN
)

In [18]:
import tensorflow as tf

In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(10000).batch(32)

2026-02-26 22:23:01.837913: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2026-02-26 22:23:01.837976: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-02-26 22:23:01.837981: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2026-02-26 22:23:01.838151: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-26 22:23:01.838286: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(32)

In [21]:
from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

# Ensure all layers are trainable (Full fine-tuning)
for layer in model.layers:
    layer.trainable = True

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
!pip install transformers

In [27]:
from transformers import create_optimizer
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=len(train_dataset) * 3  # Assuming 3 epochs
)

In [28]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [29]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

In [31]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [32]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=3,
    callbacks=[early_stopping]
)

Epoch 1/3
1250/1250 [==============================] - 3267s 3s/step - loss: 0.2848 - accuracy: 0.8791 - val_loss: 0.2503 - val_accuracy: 0.8952
Epoch 2/3
1250/1250 [==============================] - 2881s 2s/step - loss: 0.1669 - accuracy: 0.9367 - val_loss: 0.2617 - val_accuracy: 0.9001
Epoch 3/3
1250/1250 [==============================] - 2841s 2s/step - loss: 0.0881 - accuracy: 0.9695 - val_loss: 0.3094 - val_accuracy: 0.9013


In [34]:
model.evaluate(test_dataset)

313/313 [==============================] - 166s 531ms/step - loss: 0.2503 - accuracy: 0.8952


[0.2502765953540802, 0.8952000141143799]

In [35]:
from transformers import TFAutoModel

In [36]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be 

In [37]:
input_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')

In [39]:
model.outputs = model(input_ids, attention_mask=attention_mask)
cls_token = model.outputs.last_hidden_state[:, 0, :]



In [40]:
x = tf.keras.layers.Dropout(0.3)(cls_token)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [41]:
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

In [42]:
model.trainable = True

In [45]:
from transformers import create_optimizer
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=len(train_dataset) * 3  # Assuming 3 epochs
)

In [46]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [47]:
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=3,
    callbacks=[early_stopping]
)

Epoch 1/3


: 

In [ ]:
model.evaluate(test_dataset)